In [3]:
import tensorflow as tf

print (tf.version)

number = tf.Variable(324, tf.int16)

print (number.shape)

# each tensor represents a partially defined computation that will eventually produce a value. each tensor has datatype and shape - has numbers. scalar is one number

rank1_tensor = tf.Variable(["Test"], tf.string)
rank2_tensor = tf.Variable([["test", "ok", "abc"], ["test", "yes", 'abc']], tf.string)

tf.rank(rank2_tensor)

<module 'tensorflow._api.v2.version' from '/Users/tobiaslange/opt/anaconda3/lib/python3.7/site-packages/tensorflow/_api/v2/version/__init__.py'>
()


<tf.Tensor: shape=(), dtype=int32, numpy=2>

In [4]:
rank2_tensor.shape

TensorShape([2, 3])

In [7]:
tensor1 = tf.ones([1,2,3])
tensor2 = tf.reshape(tensor1, [2, 3, 1])
tensor3 = tf.reshape(tensor2, [3, -1])
print (tensor1)
print (tensor2)
print (tensor3)

tf.Tensor(
[[[1. 1. 1.]
  [1. 1. 1.]]], shape=(1, 2, 3), dtype=float32)
tf.Tensor(
[[[1.]
  [1.]
  [1.]]

 [[1.]
  [1.]
  [1.]]], shape=(2, 3, 1), dtype=float32)
tf.Tensor(
[[1. 1.]
 [1. 1.]
 [1. 1.]], shape=(3, 2), dtype=float32)


In [ ]:
tensor2.shape

In [ ]:
t = tf.zeros([5, 5, 5, 5])
print (t)

In [ ]:
t = tf.reshape(t, [125, -1])
print (t)

In [ ]:
# core ML algos - linear regression, classification, clustering, hidden markov models

import matplotlib.pyplot as plt
import numpy as np
x = [1, 2, 3, 4, 5]
y = [2, 4, 5.4, 7.2, 9]
plt.plot(x, y, "ro")

In [ ]:
 !pip install -q sklearn

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.compat.v2.feature_column as fc
data =pd.read_csv('hw4_trainingset.csv')
print (data)
# Importing SKlearn's  package 
from sklearn.model_selection import train_test_split

# this splits the data 
train, test = train_test_split(data, test_size = .2)
print (train)
print (test)
y_train = train.pop('Label')
print (y_train)
y_eval = test.pop("Label")
print (train.shape)
train.describe()
train.Feature_1.hist(bins = 20)

# things

feature_columns = []
NUMERIC_COLUMNS = ['Feature_1', "Feature_2", "Feature_3",'Feature_4', "Feature_5", "Feature_6"]
for feature_name in NUMERIC_COLUMNS:
  feature_columns.append(tf.feature_column.numeric_column(feature_name, dtype=tf.float32))

print(feature_columns)



# training computer suppose you have crazy dataset, laod into batches, give 32 entries at once to the model
# epoch - pass data to model, pretty bad, feed data again in diff order, model look at it in diff way

def make_input_fn(data_df, label_df, num_epochs=10, shuffle=True, batch_size=32):
  def input_function():
    ds = tf.data.Dataset.from_tensor_slices((dict(data_df), label_df))
    if shuffle:
      ds = ds.shuffle(1000)
    ds = ds.batch(batch_size).repeat(num_epochs)
    return ds
  return input_function

train_input_fn = make_input_fn(train, y_train)  # here we will call the input_function that was returned to us to get a dataset object we can feed to the model
eval_input_fn = make_input_fn(eval, y_eval, num_epochs=1, shuffle=False)

linear_est = tf.estimator.LinearClassifier(feature_columns = feature_columns)
linear_est.train(train_input_fn)
result = linear_est.evaluate(eval_input_fn)

print (result['accuracy'])

In [ ]:
### input layer is data we have, then take outputs as inputs for another layer, another
### until we decide to stop, last we build is output layer, compare targets to!
### first layer is input layer, last layer is output layer, layers between are hidden layers
### each hidden unit is element of tensor, number of nodes in layer is referred to as width of layer
### depth refers to number of hidden layers in a network, when we create an ML algo pick width/depth as hyperparameters
### parameters are weights/biases, h yperparameters is width/depth/learning rate
### hyperparamaters preset
### combine inputs linearly, then apply them non-linearily
### non-linearity doesn't change shape of expression, if 8 input layers predict 9, 72 weights, 72 arrows!


### prepreocessing applies to dataset through model

# feature scaling/normalization

# MNIST DATASET - 70k handwritten digits, have 10 digits, 10 classes (0-9)

# takes as input, determines which number is shown!

# MNIST is the hello world of machine learning, each image is 28-28 pixels, 28x28 matrix where input values are 0--?255
# 0 is black, 255 is white, 784 pixels, flatten - put in 784 inputs!
# 2 hidden layers, 10 classes, 10 output units!
# then compared to targets, use one-hot encoding for vectors...
# use softmax activation function for the layer

## deep neural network for MNIST

import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds

# DATA

mnist_dataset, mnist_info = tfds.load(name="mnist", with_info=True, as_supervised=True)

# split train/test
mnist_train, mnist_test = mnist_dataset["train"], mnist_dataset["test"]
# where is validation?
num_validation_samples = 0.1 * mnist_info.splits["train"].num_examples
num_validation_samples = tf.cast(num_validation_samples, tf.int64)

num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)

# scale data in some way to make result mroe numerically stable

def scale(image, label):
    image = tf.cast(image, tf.float32)
    image = image / 255
    return image, label

scaled_train_and_validation_data = mnist_train.map(scale)

test_data = mnist_test.map(scale)

# shuffle data, get valiation dataset

BUFFER_SIZE = 10000 #dealing w/ enormous datasets, cannot shuffle all at once

shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)

validation_data = shuffled_train_and_validation_data.take(num_validation_samples)
train_data = shuffled_train_and_validation_data.skip(num_validation_samples)


BATCH_SIZE = 100
train_data = train_data.batch(BATCH_SIZE)

validation_data = validation_data.batch(num_validation_samples)
test_data = test_data.batch(num_test_samples)

validation_inputs, validation_targets = next(iter(validation_data))

# Outline model
input_size = 784
output_size = 10
hidden_layer_size = 50

model = tf.keras.Sequential([
                            tf.keras.layers.Flatten(input_shape=(28,28,1)),
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                            tf.keras.layers.Dense(output_size, activation='softmax')
                            ])
# choose optimizer and the loss function

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# training
NUM_EPOCHS = 5

model.fit(train_data, epochs = NUM_EPOCHS, validation_data = (validation_inputs, validation_targets), verbose=2)

test_loss, test_accuracy = model.evaluate(test_data)

In [ ]:
def scale(image, label):
    image = tf.cast(image, tf.float32)
    return image / 255, label

scaled_train_and_validation_data = mnist_train.map(scale)

test_data = mnist_test.map(scale)

# shuffle data, get valiation dataset

BUFFER_SIZE = 10000 #dealing w/ enormous datasets, cannot shuffle all at once

shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)

validation_data = shuffled_train_and_validation_data.take(num_validation_samples)
train_data = shuffled_train_and_validation_data.skip(num_validation_samples)


BATCH_SIZE = 100
train_data = train_data.batch(BATCH_SIZE)

validation_data = validation_data.batch(num_validation_samples)
test_data = test_data.batch(num_test_samples)

validation_inputs, validation_targets = next(iter(validation_data))

# Outline model
input_size = 784
output_size = 10
hidden_layer_size = 50

model = tf.keras.Sequential([
                            tf.keras.layers.Flatten(input_shape=(28,28,1)),
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                            tf.keras.layers.Dense(output_size, activation='softmax')
                            ])
# choose optimizer and the loss function

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# training
NUM_EPOCHS = 5

model.fit(train_data, epochs = NUM_EPOCHS, validation_data = (validation_inputs, validation_targets), verbose=2)

test_loss, test_accuracy = model.evaluate(test_data)